Using a ACO to learn mywayhome (test where I limit it to 4 directions)

Import everything

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import random
import math
from torch.utils.tensorboard import SummaryWriter
from vizdoom import * #Import all of vizdoom
import time #To make the program sleep (wait), so we can actually see what's happening
from gymnasium import Env #Import OpenAI Gym's Env class
from gymnasium.spaces import Discrete, Box #Import OpenAI Gym's Discrete and Box spaces
import cv2 #OpenCV for image processing, used for modifying the DOOM environment to make it run faster 
from stable_baselines3.common.callbacks import BaseCallback #Import the BaseCallback class from stable_baselines3 to learn from the environment
from stable_baselines3.common import env_checker #Import the env_checker class from stable_baselines3 to check the environment
import os #To create directories for saving models
import sys #To change the path so we can import the pathfinder module
import matplotlib.pyplot as plt
from collections import deque

original_sys_path = sys.path.copy() #Come back to this path later after we navigate to the parent directory
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))  #Add the parent directory to the path so we can import the pathfinder module
from pathfinder import doomfinder, create_new_best_generation_directory, gamefinder #Import functions from the pathfinder module
sys.path = original_sys_path #Set the path back to the original path

Create a grid with map boundries defined 

In [ ]:
#Map extreme vertices
MIN_X, MAX_X = 160, 1120
MIN_Y, MAX_Y = -704, 128

#Grid dimensions, the map size is actually 960x832
GRID_WIDTH = 960  #1 unit per cell
GRID_HEIGHT = 832  #1 unit per cell

grid_size = (GRID_HEIGHT, GRID_WIDTH)

#Extracted vertices (x, y) coordinates
vertices = [
    (160.000, -256.000), (160.000, -96.000), (320.000, -96.000), (320.000, -144.000),
    (320.000, -256.000), (320.000, -208.000), (544.000, -256.000), (496.000, -256.000),
    (544.000, -96.000), (544.000, -144.000), (384.000, -96.000), (432.000, -96.000),
    (384.000, -256.000), (384.000, -208.000), (544.000, -32.000), (496.000, -32.000),
    (544.000, 128.000), (544.000, 80.000), (384.000, 128.000), (384.000, -32.000),
    (608.000, -256.000), (608.000, -208.000), (768.000, -256.000), (768.000, -96.000),
    (608.000, -96.000), (656.000, -96.000), (608.000, -32.000), (608.000, 16.000),
    (768.000, -32.000), (720.000, -32.000), (768.000, 128.000), (768.000, 80.000),
    (608.000, 128.000), (384.000, -384.000), (432.000, -384.000), (384.000, -544.000),
    (544.000, -544.000), (496.000, -544.000), (544.000, -384.000), (960.000, -32.000),
    (960.000, 16.000), (1120.000, -32.000), (1072.000, -32.000), (1120.000, 128.000),
    (960.000, 128.000), (960.000, -256.000), (960.000, -96.000), (1120.000, -256.000),
    (1088.000, -256.000), (1120.000, -96.000), (1008.000, -96.000), (992.000, -416.000),
    (992.000, -256.000), (1088.000, -416.000), (1072.000, -96.000), (1008.000, -32.000),
    (384.000, -144.000), (848.000, 16.000), (768.000, 16.000), (960.000, 80.000),
    (576.000, 16.000), (544.000, 16.000), (608.000, 80.000), (720.000, -96.000),
    (656.000, -32.000), (496.000, -96.000), (432.000, -32.000), (432.000, -256.000),
    (496.000, -384.000), (432.000, -704.000), (432.000, -544.000), (496.000, -640.000),
    (576.000, -640.000), (576.000, -704.000), (544.000, -208.000), (608.000, -144.000)
]

#Extracted linedefs as pairs of vertex indices
linedefs = [
    (0, 1), (1, 2), (2, 3), (4, 0), (5, 4), (6, 7), (8, 9), (10, 11),
    (12, 13), (14, 15), (16, 17), (18, 16), (19, 18), (20, 21), (22, 20),
    (23, 22), (24, 25), (26, 27), (28, 29), (30, 31), (32, 30), (33, 34),
    (35, 33), (36, 37), (38, 36), (39, 40), (41, 42), (43, 41), (44, 43),
    (45, 46), (47, 48), (49, 47), (46, 50), (51, 52), (53, 51), (48, 53),
    (48, 52), (13, 5), (52, 45), (54, 49), (42, 54), (42, 55), (50, 55),
    (50, 54), (13, 56), (55, 39), (57, 58), (31, 58), (31, 59), (40, 59),
    (40, 57), (58, 28), (59, 44), (60, 61), (17, 61), (17, 62), (27, 62),
    (27, 60), (61, 14), (62, 32), (29, 63), (29, 64), (25, 64), (25, 63),
    (56, 10), (63, 23), (64, 26), (15, 65), (15, 66), (11, 66), (11, 65),
    (3, 5), (65, 8), (66, 19), (67, 12), (34, 67), (34, 68), (7, 68),
    (7, 67), (3, 56), (68, 38), (69, 70), (37, 70), (37, 71), (71, 72),
    (72, 73), (73, 69), (70, 35), (21, 74), (9, 74), (9, 75), (21, 75),
    (75, 24), (74, 6)
]

#Initialize the grid with -1 (outside cells)
starter_grid = np.full(grid_size, -1, dtype=np.float64)

def coord_to_grid(x, y):
    #Convert coordinates to grid indices
    if x < MIN_X or x > MAX_X or y < MIN_Y or y > MAX_Y:
        raise ValueError(f"Coordinates ({x}, {y}) are out of bounds!")
    
    norm_x = (x - MIN_X) / (MAX_X - MIN_X)
    norm_y = (y - MIN_Y) / (MAX_Y - MIN_Y)
    
    grid_x = int(norm_x * (GRID_WIDTH - 1))
    grid_y = int(norm_y * (GRID_HEIGHT - 1))

    return (grid_x, grid_y)

def draw_line(starter_grid, start, end):
    x1, y1 = coord_to_grid(start[0], start[1])
    x2, y2 = coord_to_grid(end[0], end[1])
    dx = abs(x2 - x1)
    dy = abs(y2 - y1)
    sx = 1 if x1 < x2 else -1
    sy = 1 if y1 < y2 else -1
    err = dx - dy

    while True:
        starter_grid[y1, x1] = 0  # Mark cell as 0 (inside the linedef)
        if x1 == x2 and y1 == y2:
            break
        e2 = err * 2
        if e2 > -dy:
            err -= dy
            x1 += sx
        if e2 < dx:
            err += dx
            y1 += sy


#Draw all lines on the grid
for linedef in linedefs:
    start_vertex = vertices[linedef[0]]
    end_vertex = vertices[linedef[1]]
    draw_line(starter_grid, start_vertex, end_vertex)


#Function to print the grid
def print_grid(starter_grid):
    for row in starter_grid:
        print(' '.join(f'{cell:3}' for cell in row))

#Print the grid to check the result
print_grid(starter_grid)


def flood_fill(starter_grid, start_x, start_y):
    if starter_grid[start_y, start_x] != -1:  # Start only if the cell is not filled or part of the boundary
        return

    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]
    queue = deque([(start_x, start_y)])

    while queue:
        x, y = queue.popleft()

        if starter_grid[y, x] != -1:  # If already visited or is a boundary, skip
            continue

        starter_grid[y, x] = 0  # Fill the cell

        for dx, dy in directions:
            nx, ny = x + dx, y + dy
            if 0 <= nx < GRID_WIDTH and 0 <= ny < GRID_HEIGHT and starter_grid[ny, nx] == -1:
                queue.append((nx, ny))


insides = [(385, -144), (319, -208), (321, -207), (545, -207), (654, -97), (496, -641), (432, -543), (464, -320), (461, 49), (464, -68),
           (578, 48), (672, -64), (832, 64), (672, 64), (1056, -288), (1024, -160), (1056, -65), (1024, 32)]

for vertex in insides:
    x, y = coord_to_grid(vertex[0], vertex[1])
    flood_fill(starter_grid, x, y)
       

#Plot vertices
plt.figure(figsize=(10, 8))
plt.scatter(*zip(*[coord_to_grid(v[0], v[1]) for v in vertices]), c='blue', label='Vertices')

#Plot lines
for linedef in linedefs:
    start_vertex = vertices[linedef[0]]
    end_vertex = vertices[linedef[1]]
    start_grid = coord_to_grid(start_vertex[0], start_vertex[1])
    end_grid = coord_to_grid(end_vertex[0], end_vertex[1])
    plt.plot([start_grid[0], end_grid[0]], [start_grid[1], end_grid[1]], c='green')

#Create a heatmap to visualize the grid
plt.imshow(starter_grid, cmap='hot', interpolation='nearest')
plt.colorbar(label='Cell Values')
plt.title('Heatmap of Grid with Vertices and Lines')
plt.legend()
plt.show()

def get_starting_grid():
    starter_grid = np.full(grid_size, -1, dtype=np.float64)
    for linedef in linedefs:
        start_vertex = vertices[linedef[0]]
        end_vertex = vertices[linedef[1]]
        draw_line(starter_grid, start_vertex, end_vertex)
    for vertex in insides:
        x, y = coord_to_grid(vertex[0], vertex[1])
        flood_fill(starter_grid, x, y)
    return starter_grid

Define enviornment (for mywayhome)

In [3]:
#Note we are using are representing the map as a 960x832 grid

#Map extreme verticies
MIN_X, MAX_X = 160, 1120
MIN_Y, MAX_Y = -704, 128

#Grid dimensions, the map size is actually 960x832 but this allows us some leeway if we get close to the edge
GRID_WIDTH = 960 #1 unit per cell
GRID_HEIGHT = 832 #1 units per cell

starter_grid = get_starting_grid()

class mywayhome_VZG(Env): #Used for mywayhome config
    def __init__(self, config_path, render=False, grid_size=(GRID_HEIGHT, GRID_WIDTH), pheromone_evaporation_rate=0.001, pheromone_map = None, frameskip = 1): #Constructor
        #Args:
            #config_path (str): The path to the configuration file
            #render (bool): Whether to render the environment or not, false by default
            #grid_size (tuple): The size of the grid (x, y) for the environment
            #pheromone_evaporation_rate (float): The rate at which pheromones evaporate over time
            #pheromone_map (np.array): Optional pre-filled pheromone map, if not provided, it will be initialized with low pheromone levels
            #frameskip (int): The number of frames to skip between actions

        super(mywayhome_VZG, self).__init__() #Inherit from Env class

        #Setup game environment
        self.game = vizdoom.DoomGame() #Create a DoomGame object
        self.game.set_doom_game_path(gamefinder('DOOM2.WAD')) #Set the path to the game
        self.game.load_config(config_path) #Load the configuration file from file path
        self.game.set_window_visible(render) #Set window visibility based on render argument
        self.game.init() #Initialize the game
        self.frameskip = frameskip #Set the frameskip
        
        #Setup action and observation space
        self.observation_space = Box(low=0, high=255, shape=(100, 160, 1), dtype=np.uint8) #Observation space, 100x160x1 image
        self.action_space = Discrete(4) #Action space, 4 actions (move forward, move left, move right, move backwards)

        #Grid dimensions and pheromone setup
        self.grid_size = grid_size #Set the grid size as specified 
        if pheromone_map is not None:
            self.pheromone_map = pheromone_map #Use the provided pheromone map if available
        else: #If no pheromone map is provided, initialize a new one
            self.pheromone_map = starter_grid  #Start with adjustable pheromone levels across the gri
        self.pheromone_evaporation_rate = pheromone_evaporation_rate  #Controls how fast pheromones decay over time

        #Get game variables
        game_variables = self.game.get_state().game_variables
        xpos, ypos, angle, velocity_x, velocity_y = game_variables #Unpack the game variables
        
        self.xpos = xpos #X position of the player
        self.ypos = ypos #Y position of the player
        self.angle = angle #Angle of the player 
        self.velocity_x = velocity_x #X velocity of the player
        self.velocity_y = velocity_y #Y velocity of the player

    def coord_to_grid(self, x, y): #Convert coordinates to grid indices
        #Args:
            #x (int): X coordinate to convert
            #y (int): Y coordinate to convert
        #Returns:
            #grid_x, grid_y (tuple): The grid indices corresponding to the coordinates
        #Raises:
            #ValueError: If the coordinates are out of bounds

        #Ensure coordinates are within world bounds
        #if x < MIN_X or x > MAX_X or y < MIN_Y or y > MAX_Y:
        #    raise ValueError(f"Coordinates ({x}, {y}) are out of bounds!")
        
        #Normalize coordinates to range [0, 1]
        norm_x = (x - MIN_X) / (MAX_X - MIN_X)
        norm_y = (y - MIN_Y) / (MAX_Y - MIN_Y)
        
        #print(f"norm_x = {norm_x}, norm_y = {norm_y}") #Debug print

        #Scale to grid dimensions and floor the values to ensure integers
        grid_x = int(norm_x * (GRID_WIDTH - 1))
        grid_y = int(norm_y * (GRID_HEIGHT - 1))

        #print(f"grid_x = {grid_x}, grid_y = {grid_y}") #Debug print

        return (grid_x, grid_y)

    def step(self, action, limit = 1000): #Take a step in the environment 
        #Args:
            #action (int): The action to take
            #limit (int): Unimplemented "limit" for the episode, most likely will be a time limit
        #Returns:
            #observation (np.array): The screen buffer of the environment
            #reward (float): The reward for the action taken
            #terminated (bool) Whether the episode is finished or not (by reaching the goal)
            #truncated (bool): Whether the episode has reached some terminal state without reaching the goal (ie: running out of time)
            #info (dict): Additional information about the environment

        #print(f"xpos = {self.xpos}, ypos = {self.ypos}, angle = {self.angle}") #Debug print
        #time.sleep(1) #DEBUG -----------------------------------

        #Specify actions and take a step
        actions = np.identity(4) #Create an identity matrix with 4 rows (4 actions), TURN_LEFT, TURN_RIGHT, MOVE_FOWARD, MOVE_LEFT, MOVE_RIGHT, MOVE_BACKWARD, these are the actions we can take in the environment

        self.game.make_action(actions[action], self.frameskip) #Take an action in the action space, second parameter is frame skip (how many frames we take an action for)
        truncated = False #Not implemented yet, so set to False. The idea is that if step passes some sort of limit, like a time limit, then the episode is truncated.
        info = {"xpos": self.xpos, "ypos": self.ypos, "angle": self.angle, "angle": self.angle, "velocity_x": self.velocity_x, "velocity_y": self.velocity_y} #Initialize info with current position and angle
        reward = 0 #Initialize reward to 0

        if self.game.get_state(): #If the game is not finished
            observation = self.game.get_state().screen_buffer #Get the screen buffer
            observation = self.greyscale(observation) #Convert the image to greyscale
            
            xpos, ypos, angle, velocity_x, velocity_y = self.game.get_state().game_variables #Get the game variables
            self.xpos = xpos #Update the x position
            self.ypos = ypos #Update the y position
            self.angle = angle #Update the angle
            self.velocity_x = velocity_x #Update the x velocity
            self.velocity_y = velocity_y #Update the y velocity

            info = {"xpos": self.xpos, "ypos": self.ypos, "angle": self.angle, "velocity_x": self.velocity_x, "velocity_y": self.velocity_y} #Add position, angle and velocity info to the info dictionary

        #Update pheromones based on movement feedback
            #If this isnt a wall, add pheromones
            #self.pheromone_map[grid_y, grid_x] += 0.1 #Deposit pheromones
            #self.pheromone_map[self.pheromone_map != -1] *= (1 - self.pheromone_evaporation_rate)  #Simulate evaporation, except on walls
        else: #If the game is finished
            observation = np.zeros(self.observation_space.shape) #Return a blank screen

        terminated = self.game.is_episode_finished() #Check if the episode is finished

        return observation, reward, terminated, truncated, info

    def reset(self): #Reset the environment when we start a new game
        #Args:
            #seed (int): The seed for the random number generator
        #Returns:
            #(observation, info) (tuple)
                #observation (np.array): The screen buffer of the environment
                #info (dict): Additional information about the environment
                
        self.game.new_episode() #Start a new episode
        game_variables = self.game.get_state().game_variables  #Get the game variables   
        self.xpos, self.ypos, self.angle, self.velocity_x, self.velocity_y = game_variables  #Unpack the game variables, set agent to its random starting position 

        state = self.game.get_state().screen_buffer
        observation = self.greyscale(state)  #Convert the initial screen state to greyscale
        info = {"xpos": self.xpos, "ypos": self.ypos, "angle": self.angle, "velocity_x": self.velocity_x, "velocity_y": self.velocity_y} #Add position, angle and velocity info to the info dictionary

        return observation, info  #Return the observation and any additional info

    def greyscale(self, observation): #Convert the environment image to greyscale and resize it
        #Args:
            #observation (np.array): The image of the environment (the current game frame)
        #Returns:
            #grey_return (np.array): The resized greyscale image of the environment
        
        grey = cv2.cvtColor(np.moveaxis(observation, 0, -1), cv2.COLOR_BGR2GRAY) #Convert the image to greyscale
        resize = cv2.resize(grey, (160, 100), interpolation=cv2.INTER_CUBIC) #Resize the image to 160x100 (frame size)
        state = np.reshape(resize, (100, 160, 1)) #Reshape to 100x160x1 for compatibility with neural networks
        
        return state

    def render(self, render_in_greyscale=False): #Render the environment for a frame
        #Args:
            #render_in_greyscale (bool): Whether to render the environment in greyscale or not
        
        if self.game.get_state() and render_in_greyscale:  #Only render if there's a valid game state
            observation = self.game.get_state().screen_buffer
            greyscale_obs = self.greyscale(observation)  #Convert the observation to greyscale
            cv2.imshow("VizDoom Environment", greyscale_obs.squeeze())  #Remove extra dimension and display
            cv2.waitKey(1)  #Wait 1ms between frames to allow for rendering
        elif self.game.get_state():  #Only render if there's a valid game state
            observation = self.game.get_state().screen_buffer
            cv2.imshow("VizDoom Environment", observation.squeeze())  #Render the environment without greyscale
            cv2.waitKey(1)  #Wait 1ms between frames to allow for rendering
        else:
            print("No game state to render.")

    def get_state(self): #Get the current state of the environment
        #Returns:
            #state (np.array): The current state of the environment
        return self.game.get_state()

    def get_agent_position(self): #Get the current position of the agent (tuple of x, y coordinates)
        #Returns:
            #(xpos, ypos) (tuple): The current position of the agent

        return (self.xpos, self.ypos)

    def close(self): #Close the environment when done
        self.game.close()  #Terminate the game

Define Plotting, Saving, and Finding Pheromone Maps

In [4]:
import matplotlib.colors as mcolors

def plot_pheromone_grid(pheromone_grid, title, log_dir, iteration, save_image):
    #Create a custom colormap
    cmap = plt.cm.hot
    cmap = cmap.copy()
    cmap.set_bad(color='blue') #Set color for walls (-1) to black

    #Mask the grid to handle walls (-1)
    masked_grid = np.ma.masked_where(pheromone_grid == -1, pheromone_grid)

    plt.imshow(masked_grid, cmap=cmap, interpolation='nearest')
    plt.title(title)
    plt.colorbar(label="Pheromone Intensity")
    if save_image:
        image_path = os.path.join(log_dir, f"pheromone_grid_iteration_{iteration}.png")
        plt.savefig(image_path)
        print(f"Pheromone grid image saved to {image_path}")
    plt.show()


def create_run_directory(base_dir="aco_runs/vizdoom_aco_mywayhome"): #Create a new run directory for the ACO algorithm
    os.makedirs(base_dir, exist_ok=True)
    existing_runs = [int(d.split('_')[-1]) for d in os.listdir(base_dir) if d.split('_')[-1].isdigit()]
    run_number = max(existing_runs, default=0) + 1
    run_dir = os.path.join(base_dir, f"run_{run_number}")
    os.makedirs(run_dir, exist_ok=True)
    #Create saved maps directory within the run directory
    maps_dir = os.path.join(run_dir, "saved_maps")
    os.makedirs(maps_dir, exist_ok=True)
    #Create log directory within the run directory
    log_dir = os.path.join(run_dir, "log")
    os.makedirs(log_dir, exist_ok=True)
    return run_dir, log_dir, maps_dir

def save_pheromone_grid(pheromone_grid, maps_dir, iteration):
    filename = os.path.join(maps_dir, f"pheromone_grid_iteration_{iteration}.npy")
    np.save(filename, pheromone_grid)
    #print(f"Pheromone grid saved to {filename}")

def load_pheromone_grid(run_dir="aco_runs/vizdoom_aco_mywayhome", run_number=0, filename="pheromone_grid.npy"):
    run_dir = os.path.join(run_dir, f"run_{run_number}/saved_maps")
    filepath = os.path.join(run_dir, filename)
    #print(filepath) #Print the path
    if os.path.exists(filepath):
        pheromone_grid = np.load(filepath)
        #print(f"Pheromone grid loaded from {filepath}")
        return pheromone_grid
    else:
        #print(f"No saved pheromone grid found at {filepath}")
        return None

Define Training

In [5]:
def aco_training(env, num_ants=10, iterations=50, alpha=0.6, beta=0.4, pheromone_deposit=2.0, unsuccessful_deposit=0.1, run_dir="aco_runs/vizdoom_aco_mywayhome", log_dir = "aco_runs/vizdoom_aco_mywayhome/run_x/log", maps_dir = "aco_runs/vizdoom_aco_mywayhome/run_x/saved_maps", distance_weight = 0.7, time_weight = 0.3): #Train the ACO algorithm in the ViZDoom environment
    """
    Train a bot to pathfind in the ViZDoom environment using ACO.
    
    Args:
        env (Env): The ViZDoom environment.
        num_ants (int): Number of ants (pathfinding agents) per iteration.
        iterations (int): Total iterations for ACO.
        alpha (float): Importance of pheromones.
        beta (float): Importance of heuristic (distance to goal).
        pheromone_deposit (float): Amount of pheromone deposited on successful paths.
        unsuccessful_deposit (float): Amount of pheromone deposited on unsuccessful paths.
        run_dir (str): Directory to save the ACO run data.
        log_dir (str): Directory to save logs and images.
        maps_dir (str): Directory to save pheromone maps.
        distance_weight (float): Weight for distance factor in pheromone update.
        time_weight (float): Weight for time factor in pheromone update.    """
    #Initialize pheromone grid
    pheromone_grid = env.pheromone_map
    
    frameskip = env.frameskip #Get the frameskip from the environment

    #Normalize alpha and beta to ensure they add up to 1
    total_weight = alpha + beta
    alpha /= total_weight
    beta /= total_weight

    #Generate list of goal positions
    #GOAL_POSITIONS = [env.coord_to_grid(x, y) for x in range(1008, 1072) for y in range(-324, -315)] 

    GOAL_POSITION = (1045, -316) #We simplify from a list of goal positions to one goal position because 
    #having a list of goal positions causes the agent to get stuck in a loop where it keeps going back and forth 
    #between the goal positions different equal minimum distances, which weakens the hueristic function, this 
    #position will always trigger the end of the episode

    #Heuristic function: inverse distance to the goal
    def heuristic(x, y):
        gx, gy = GOAL_POSITION
        distance = np.sqrt((gx - x) ** 2 + (gy - y) ** 2)
        return 1.0 / (distance + 1e-6)

    #Simulate movement locally without stepping the game
    def simulate_move(x, y, angle, velocity_x, velocity_y, move):   
        #print(f"x passed is {x}, y passed is {y}")  #Debug print

        step_size = env.frameskip  #Approximate movement step size, this varies HEAVILY based on momentum and such, not sure how to get an accurate approximation each time
        angle_rad = np.deg2rad(angle)  #Convert angle to radians

        if move == 0:  #MOVE_FORWARD
            new_x = x + step_size * (np.cos(angle_rad) + velocity_x)
            new_y = y + step_size * (np.sin(angle_rad) + velocity_y)
        elif move == 1:  #MOVE_LEFT
            new_x = x - step_size * (np.sin(angle_rad) + velocity_x)
            new_y = y + step_size * (np.cos(angle_rad) + velocity_y)
        elif move == 2:  #MOVE_RIGHT
            new_x = x + step_size * (np.sin(angle_rad) + velocity_x)
            new_y = y - step_size * (np.cos(angle_rad) + velocity_y)
        elif move == 3:  #MOVE_BACKWARD
            new_x = x - step_size * (np.cos(angle_rad) + velocity_x)
            new_y = y - step_size * (np.sin(angle_rad) + velocity_y)
        else:
            new_x, new_y = x, y

        return new_x, new_y

    #Function to select next move based on pheromones and heuristic
    def select_next_move(x, y, x_velocity, y_velocity, angle):
        possible_moves = [0, 1, 2, 3]  #MOVE_FORWARD, MOVE_LEFT, MOVE_RIGHT, MOVE_BACKWARD
        probs = []
        backup_probs = [] #Saves probabilities incase all moves have already been taken
        valid_moves = [] #List of valid moves
        pheromone_values = [] #List of pheromone values in the possible moves
        eta_values = [] #List of heuristic values in the possible moves
        distances = [] #List of distances to the goal for each move

        for move in possible_moves:
            #Simulate movement locally without stepping the game
            simulated_x, simulated_y = simulate_move(x, y, x_velocity, y_velocity, angle, move)
            #print(f"Simulated move: {move}, x: {simulated_x}, y: {simulated_y}, angle: {simulated_angle}")  #Debug print

            #Convert to grid indices and calculate pheromone + heuristic
            grid_x, grid_y = env.coord_to_grid(simulated_x, simulated_y)
            if 0 <= grid_x < GRID_WIDTH and 0 <= grid_y < GRID_HEIGHT:
                pheromone = pheromone_grid[grid_y, grid_x]
                if pheromone > 0: #If the move is not a wall
                    eta = heuristic(grid_x, grid_y)
                    distance = np.sqrt((GOAL_POSITION[0] - grid_x) ** 2 + (GOAL_POSITION[1] - grid_y) ** 2)
                    #print(f"eta = {eta}, pheromone = {pheromone}")
                    pheromone_values.append(pheromone)
                    eta_values.append(eta)
                    distances.append(distance)
                    valid_moves.append((move, grid_x, grid_y)) #Add the move to the list of valid moves

        if not valid_moves:
            #print("No valid moves available. Selecting a random move.")
            return np.random.choice(possible_moves)

        max_pheromone = np.max(pheromone_values) #Get the maximum pheromone level
        max_eta = np.max(eta_values) #Get the maximum eta value

        #Rank the moves based on distance to the goal
        sorted_indices = np.argsort(distances)
        valid_moves = [valid_moves[i] for i in sorted_indices]
        rank_boosts = np.zeros(len(valid_moves))
        boost_values = [1.0, 0.5, 0.25, 0.0]  #The move closest to the goal gets the highest boost, the farthest gets the lowest
        moves_considered = []

        rank = 0 #Start at rank 0
        for i in range(len(sorted_indices)): #Iterate through the sorted indices
            if i > 0 and distances[sorted_indices[i]] != distances[sorted_indices[i-1]]: #If the distance is different from the previous distance
                rank += 1 #Increment the rank 
            if rank < len(boost_values): #If the rank is less than the number of boost values
                rank_boosts[sorted_indices[i]] = boost_values[rank] #Set the rank boost value
            else:
                rank_boosts[sorted_indices[i]] = 0.0 #Otherwise, set the rank boost to 0

        for i, (move, grid_x, grid_y) in enumerate(valid_moves):
            pheromone = pheromone_grid[grid_y, grid_x]
            eta = heuristic(grid_x, grid_y)
            #Normalize pheromone and heuristic values
            norm_pheromone = pheromone / max_pheromone
            norm_eta = eta / max_eta  
            prob = (norm_pheromone ** alpha) * (norm_eta ** (beta * (1 + rank_boosts[i]))) #Boost the probability based on rank
            if (grid_x, grid_y) in path:
                probs.append(0)  #Set probability to zero if the move has already been made
                backup_probs.append(prob) 
            else:
                probs.append(prob)
                backup_probs.append(prob)
                moves_considered.append((move, grid_x, grid_y)) #Add the move to the list of moves considered
            #print(f"Move: {move}, pheromone: {pheromone}, eta: {eta}, norm_pheromone: {norm_pheromone}, norm_eta: {norm_eta}")

        #Normalize probabilities
        probs = np.array(probs)
        backup_probs = np.array(backup_probs)
        
        if probs.sum() == 0 and backup_probs.sum() == 0:
            probs = np.ones(len(valid_moves)) / len(valid_moves)  #Equal probability
        elif probs.sum() == 0: #If all possible moves have been made
            probs = backup_probs / backup_probs.sum()
        else:
            probs /= probs.sum()

        choice = np.random.choice([move for move, _, _ in valid_moves], p=probs)

        return choice
    
    #Define boundaries for classifying spawn locations
    def classify_spawn(x, y):
        gx, gy = GOAL_POSITION
        distance = np.sqrt((gx - x) ** 2 + (gy - y) ** 2)
        #print(f"Distance to goal: {distance}")
        if distance > 600:  # Example threshold for "far"
            return "far"
        elif distance > 400:  #Spawned in the middle of the map
            return "relatively close"
        else:  
            return "close" #Spawned in the corder facing the goal

    #Main ACO loop
    
    plot_pheromone_grid(pheromone_grid, "Initial Grid", log_dir, 0, save_image=False) #Plot the pheromone grid after each ant
    #Initialize running tally dictionaries
    total_success_counts = {
        "far": 0,
        "relatively close": 0,
        "close": 0
    }
    total_spawn_counts = {
        "far": 0,
        "relatively close": 0,
        "close": 0
    }

    #Open the file to save the success counts
    success_file_path = os.path.join(run_dir, "success_counts.txt")
    with open(success_file_path, "w") as success_file:
        for iteration in range(iterations): 
            print(f"Iteration {iteration + 1}/{iterations}")

            #Initialize counters for successful ants and spawn counts
            success_counts = {
                "far": 0,
                "relatively close": 0,
                "close": 0
            }
            spawn_counts = {
                "far": 0,
                "relatively close": 0,
                "close": 0
            }

            for ant in range(num_ants):
                
                terminated = False
                tics = 10 #Start at 10 tics

                observation, info = env.reset()
                x, y, angle, x_velocity, y_velocity = env.xpos, env.ypos, env.angle, env.velocity_x, env.velocity_y #Get the initial position and velocity of the ant
                path = []
                success = False
                spawn_classification = classify_spawn(x, y) #We spawned in this area
                spawn_counts[spawn_classification] += 1  #Increment the spawn counter

                #Calculate initial distance to the closest goal
                gx, gy = GOAL_POSITION
                initial_distance = np.sqrt((gx - x) ** 2 + (gy - y) ** 2)
                max_distance = np.sqrt(GRID_WIDTH ** 2 + GRID_HEIGHT ** 2)  #Maximum possible distance in the environment
                distance_factor = initial_distance / max_distance  #Normalize the distance factor

                while not terminated:  #Steps are limited to number of tics (2100) by the config
                    action = select_next_move(x, y, x_velocity, y_velocity, angle)
                    observation, _, terminated, _, info = env.step(action)
                    x, y, angle = info['xpos'], info['ypos'], info['angle']

                    converted_x, converted_y = env.coord_to_grid(x, y) #Convert the position to grid indices

                    #Interpolate missing points in real-time
                    if path:
                        last_x, last_y = path[-1]
                        dx = converted_x - last_x
                        dy = converted_y - last_y
                        steps = int(max(abs(dx), abs(dy)))  #Convert steps to an integer
                        if steps > 1:  #Interpolate the missing points
                            for step in range(1, steps):
                                interpolated_x = round(last_x + step * dx / steps)
                                interpolated_y = round(last_y + step * dy / steps)
                                path.append((interpolated_x, interpolated_y))

                    tics = tics + 1 * frameskip #Increment tics, multiply by frameskip to account for the frameskip
                    
                    path.append((converted_x, converted_y)) #Add the current position to the path

                if tics < 8400: #If the ant reached the goal
                    print(f"Ant {ant} reached the goal in {tics} steps.")
                    success = True
                    success_counts[spawn_classification] += 1 

                #Evaporate pheromones
                adjusted_evaporation_rate = env.pheromone_evaporation_rate * tics  #Evaporate more if the ant took longer
                pheromone_grid[pheromone_grid != -1] *= (1 - adjusted_evaporation_rate)  #Do evaporation after ant instead of in env

                #Apply pheromones
                visited_points = set()
                if success:
                    for (px, py) in path: #Give a little boost to path if the ant reached the goal
                        if (px, py) not in visited_points:
                            visited_points.add((px, py))
                            time_factor = 1.0 / (tics + 1e-6)  #Inverse of the number of tics taken
                            combined_weight = distance_factor * distance_weight + time_factor * time_weight  #Prioritize distance factor
                            pheromone_grid[py, px] += pheromone_deposit * combined_weight
                            #print(combined_weight*pheromone_deposit)
                else:
                    for (px, py) in path:
                        if (px, py) not in visited_points:
                            visited_points.add((px, py))
                            pheromone_grid[py, px] += unsuccessful_deposit #Deposit pheromones for the path taken

                print(f"Ant {iteration}.{ant}, tics: {tics}, success: {success}")
                print(f"Path: {path}")
                plot_pheromone_grid(pheromone_grid, (f"Pheromone Grid After Ant {iteration}.{ant}"), log_dir, iteration, save_image=False) #Plot the pheromone grid after each ant
            
            if(iteration % 5 == 0 and iteration < iterations): #Save the pheromone grid after every 5 iterations
                plot_pheromone_grid(pheromone_grid, (f"Iteration {iteration}"), log_dir, iteration, save_image=True)  #Save the image after every 10 iterations
            else: 
                plot_pheromone_grid(pheromone_grid, (f"Iteration {iteration}"), log_dir, iteration, save_image=False)  #Plot the pheromone grid after each iteration
            save_pheromone_grid(pheromone_grid, maps_dir, iteration)  #Save the pheromone grid at the end of iteration
        
            #Print the spawn counts and success ratios
            print("-" * 50)
            print(f"Spawn counts and success rates (for iteration {iteration}):\n")
            success_file.write(f"\nSpawn counts and success rates (for iteration {iteration}):\n")
            for classification, count in spawn_counts.items():
                print(f"Spawned {classification}: {success_counts[classification]}/{count}")   
                success_file.write(f"Spawned {classification}: {success_counts[classification]}/{count}\n")  

            for classification in total_spawn_counts.keys():
                total_spawn_counts[classification] += spawn_counts[classification]
                total_success_counts[classification] += success_counts[classification]
            
            #Print running tally
            print("Total spawn counts and success rates so far:")
            success_file.write(f"\nTotal spawn counts and success rates at iteration {iteration}/{iterations}\n")
            for classification, count in total_spawn_counts.items():
                print(f"Total spawned {classification}: {total_success_counts[classification]}/{count}")
                success_file.write(f"Total spawned {classification}: {total_success_counts[classification]}/{count}\n")
            print("-" * 50)

    plot_pheromone_grid(pheromone_grid, (f"Final Map (Iteration {iterations})"), log_dir, iteration, save_image=True)  #Plot/save the final pheromone grid
    env.close()

Enviornment to Test Maps Trained

In [6]:
def test_saved_maps(env, num_tests=5, alpha = 0.6, beta = 0.4, pheromone_grid=None):
    """
    Test the saved pheromone maps in the ViZDoom environment.
    
    Args:
        env (Env): The ViZDoom environment.
        num_tests (int): Number of test runs to perform.
        alpha (float): Importance of pheromones.
        beta (float): Importance of heuristic (distance to goal).
        pheromone_grid (np.array): The saved pheromone grid to test.
    """
    #Load the saved pheromone grid
    if pheromone_grid is None:
        print("No saved pheromone grid found.")
        return
    
    env.pheromone_map = pheromone_grid
    
    #Normalize alpha and beta to ensure they add up to 1
    total_weight = alpha + beta
    alpha /= total_weight
    beta /= total_weight

    #Generate list of goal positions
    #GOAL_POSITIONS = [env.coord_to_grid(x, y) for x in range(1008, 1072) for y in range(-324, -315)] 

    GOAL_POSITION = (1045, -316) #We simplify from a list of goal positions to one goal position because 
    #having a list of goal positions causes the agent to get stuck in a loop where it keeps going back and forth 
    #between the goal positions different equal minimum distances, which weakens the hueristic function, this 
    #position will always trigger the end of the episode

    #Heuristic function: inverse distance to the goal
    def heuristic(x, y):
        gx, gy = GOAL_POSITION
        distance = np.sqrt((gx - x) ** 2 + (gy - y) ** 2)
        return 1.0 / (distance + 1e-6)
    
    #Simulate movement locally without stepping the game
    def simulate_move(x, y, angle, velocity_x, velocity_y, move):
        #print(f"x passed is {x}, y passed is {y}")  #Debug print

        step_size = env.frameskip  #Approximate movement step size, this varies HEAVILY based on momentum and such, not sure how to get an accurate approximation each time
        angle_rad = np.deg2rad(angle)  #Convert angle to radians

        if move == 0:  #MOVE_FORWARD
            new_x = x + step_size * (np.cos(angle_rad) + velocity_x)
            new_y = y + step_size * (np.sin(angle_rad) + velocity_y)
        elif move == 1:  #MOVE_LEFT
            new_x = x - step_size * (np.sin(angle_rad) + velocity_x)
            new_y = y + step_size * (np.cos(angle_rad) + velocity_y)
        elif move == 2:  #MOVE_RIGHT
            new_x = x + step_size * (np.sin(angle_rad) + velocity_x)
            new_y = y - step_size * (np.cos(angle_rad) + velocity_y)
        elif move == 3:  #MOVE_BACKWARD
            new_x = x - step_size * (np.cos(angle_rad) + velocity_x)
            new_y = y - step_size * (np.sin(angle_rad) + velocity_y)
        else:
            new_x, new_y = x, y

        #print(f"Simulated move: {move}, x: {new_x}, y: {new_y}, angle: {angle}")  #Debug print
        print(velocity_x, velocity_y) #Debug print
        
        return new_x, new_y

    #Function to select next move based on pheromones and heuristic
    def select_next_move(x, y, velocity_x, velocity_y, angle):
        possible_moves = [0, 1, 2, 3]  #MOVE_FORWARD, MOVE_LEFT, MOVE_RIGHT, MOVE_BACKWARD
        probs = []
        backup_probs = [] #Saves probabilities incase all moves have already been taken
        valid_moves = [] #List of valid moves
        pheromone_values = [] #List of pheromone values in the possible moves
        eta_values = [] #List of heuristic values in the possible moves
        distances = [] #List of distances to the goal for each move
         
        for move in possible_moves:
            #Simulate movement locally without stepping the game
            #walls_detected = []
            simulated_x, simulated_y = simulate_move(x, y, angle, velocity_x, velocity_y, move)
            #print(f"Simulated move: {move}, x: {simulated_x}, y: {simulated_y}, angle: {simulated_angle}")  #Debug print

            #Convert to grid indices and calculate pheromone + heuristic
            grid_x, grid_y = env.coord_to_grid(simulated_x, simulated_y)
            if 0 <= grid_x < GRID_WIDTH and 0 <= grid_y < GRID_HEIGHT:
                pheromone = pheromone_grid[grid_y, grid_x]
                if pheromone > 0: #If the move is not into a wall
                    eta = heuristic(grid_x, grid_y)
                    distance = np.sqrt((GOAL_POSITION[0] - grid_x) ** 2 + (GOAL_POSITION[1] - grid_y) ** 2)
                    #print(f"eta = {eta}, pheromone = {pheromone}")
                    pheromone_values.append(pheromone)
                    eta_values.append(eta)
                    distances.append(distance)
                    valid_moves.append((move, grid_x, grid_y)) #Add the move to the list of valid moves
                #else:
                    #print(f"Wall detected, so I wont move to {env.coord_to_grid(simulated_x, simulated_y)}")  #Debug print
                    #walls_detected.append(move)

        if not valid_moves:
            #print("No valid moves available. Selecting a random move.")
            return np.random.choice(possible_moves)

        max_pheromone = np.max(pheromone_values) #Get the maximum pheromone level
        max_eta = np.max(eta_values) #Get the maximum eta value

        #Rank the moves based on distance to the goal
        sorted_indices = np.argsort(distances)
        valid_moves = [valid_moves[i] for i in sorted_indices]
        rank_boosts = np.zeros(len(valid_moves))
        boost_values = [1.0, 0.5, 0.25, 0.0]  #The move closest to the goal gets the highest boost, the farthest gets the lowest
        moves_considered = []

        #print(f"Sorted indices: {sorted_indices}")
        #print(f"Valid moves: {valid_moves}") #Debug print, check to see if the moves are sorted correctly

        rank = 0 #Start at rank 0
        for i in range(len(sorted_indices)): #Iterate through the sorted indices
            if i > 0 and distances[sorted_indices[i]] != distances[sorted_indices[i-1]]: #If the distance is different from the previous distance
                rank += 1 #Increment the rank 
            if rank < len(boost_values): #If the rank is less than the number of boost values
                rank_boosts[sorted_indices[i]] = boost_values[rank] #Set the rank boost value
            else:
                rank_boosts[sorted_indices[i]] = 0.0 #Otherwise, set the rank boost to 0

        for i, (move, grid_x, grid_y) in enumerate(valid_moves):
            pheromone = pheromone_grid[grid_y, grid_x]
            eta = heuristic(grid_x, grid_y)
            #Normalize pheromone and heuristic values
            norm_pheromone = pheromone / max_pheromone
            norm_eta = eta / max_eta  
            prob = (norm_pheromone ** alpha) * (norm_eta ** (beta * (1 + rank_boosts[i]))) #Boost the probability based on rank
            if (grid_x, grid_y) in path:
                probs.append(0)  #Set probability to zero if the move has already been made
                backup_probs.append(prob) 
            else:
                probs.append(prob)
                backup_probs.append(prob)
                moves_considered.append((move, grid_x, grid_y)) #Add the move to the list of moves considered
            #print(f"Move: {move}, pheromone: {pheromone}, eta: {eta}, norm_pheromone: {norm_pheromone}, norm_eta: {norm_eta}")

        #Normalize probabilities
        probs = np.array(probs)
        backup_probs = np.array(backup_probs)
        
        if probs.sum() == 0 and backup_probs.sum() == 0:
            probs = np.ones(len(valid_moves)) / len(valid_moves)  #Equal probability
        elif probs.sum() == 0: #If all possible moves have been made
            probs = backup_probs / backup_probs.sum()
        else:
            probs /= probs.sum()

        choice = np.random.choice([move for move, _, _ in valid_moves], p=probs)

        print(f"Moves considered: {moves_considered}")
        print(f"Choice made: {choice}")

        #This seems to not happen ever so thats good
        #if choice in walls_detected:
            #print("I ran into a wall for some reason.")
            #time.sleep(2) #Sleep for 2 seconds, so we can see what happened

        return choice

    #Define boundaries for classifying spawn locations
    def classify_spawn(x, y):
        gx, gy = GOAL_POSITION
        distance = np.sqrt((gx - x) ** 2 + (gy - y) ** 2)
        #print(f"Distance to goal: {distance}")
        if distance > 600:  #Spawned far from the goal
            return "far"
        elif distance > 400:  #Spawned in the middle of the map
            return "relatively close"
        else:  
            return "close" #Spawned in the corder facing the goal
 
    #Initialize counters for successful ants and spawn counts
    success_counts = {
        "far": 0,
        "relatively close": 0,
        "close": 0
    }
    spawn_counts = {
        "far": 0,
        "relatively close": 0,
        "close": 0
    }

    #Run test episodes
    for test in range(num_tests):
        terminated = False
        tics = 10  #Start at 10 tics

        observation, info = env.reset()
        x, y, angle, velocity_x, velocity_y = env.xpos, env.ypos, env.angle, env.velocity_x, env.velocity_y #Get the initial position and velocity of the ant
        path = []
        success = False

        spawn_classification = classify_spawn(x, y) #We spawned in this area
        spawn_counts[spawn_classification] += 1  #Increment the spawn counter

        print(f"Spawn location: ({env.coord_to_grid(x,y)[0]}, {env.coord_to_grid(x,y)[1]}) ({spawn_classification})")  #Debug print

        while not terminated:  #Steps are limited to number of tics (2100) by the config
            action = select_next_move(x, y, velocity_x, velocity_y, angle)
            observation, _, terminated, _, info = env.step(action)
            x, y, angle, velocity_x, velocity_y = info['xpos'], info['ypos'], info['angle'], info['velocity_x'], info['velocity_y']  #Get the new position and velocity of the ant
            #print(f"x = {x}, which is {env.coord_to_grid(x, y)[0]} when converted to grid, y = {y}, which is {env.coord_to_grid(x, y)[1]} when converted to grid")  #Debug print
            converted_x, converted_y = env.coord_to_grid(x, y)  #Convert to grid indices

            print(f"We ended up at {converted_x, converted_y}")  
            
            #Interpolate missing points in real-time
            if path:
                last_x, last_y = path[-1]
                dx = converted_x - last_x
                dy = converted_y - last_y
                steps = int(max(abs(dx), abs(dy)))  #Convert steps to an integer
                if steps > 1:  #Interpolate the missing points
                    for step in range(1, steps):
                        interpolated_x = round(last_x + step * dx / steps)
                        interpolated_y = round(last_y + step * dy / steps)
                        path.append((interpolated_x, interpolated_y))

            path.append((converted_x, converted_y))            
            tics = tics + 1 * env.frameskip  #Increment tics, multiply by frameskip to account for the frameskip
        
        if tics < 8400:  #If the agent reached the goal
            print(f"Test {test + 1}: Reached the goal in {tics} steps.")
            success = True
            success_counts[spawn_classification] += 1 
        else:
            print(f"Test {test + 1}: Did not reach the goal.")
        
        print(f"Test {test + 1}, tics: {tics}, success: {success}")
        print(f"Path: {path}")
        time.sleep(2)  #Sleep for 2 seconds between tests
        #plot_pheromone_grid(pheromone_grid, f"Test {test + 1}", test + 1, save_image=False)  #Plot and save the pheromone grid after each test

    #Print the spawn counts and success ratios
    print("Spawn counts and success ratios:")
    for classification, count in spawn_counts.items():
        print(f"Spawned {classification}: {success_counts[classification]}/{count}")

    env.close()

Load the Pheromone Map you want to start with

In [7]:
#Load existing pheromone grid
saved_pheromone_grid = load_pheromone_grid(run_number=10, filename='pheromone_grid_iteration_199.npy')
#saved_pheromone_grid = None

Run Training

In [ ]:
#Initialize the environment and run ACO training

env = mywayhome_VZG(config_path=doomfinder("my_way_home_4_dir.cfg"), pheromone_evaporation_rate=0.00003968253, render=False, frameskip=4) #Lower frameskip makes better maps but takes longer

#Create run directory
run_dir, log_dir, maps_dir = create_run_directory()

if saved_pheromone_grid is not None:
        env.pheromone_map = saved_pheromone_grid
else:
    env.pheromone_map = get_starting_grid()
    
aco_training(env, num_ants = 10, iterations=200, pheromone_deposit = 8.0, run_dir=run_dir, log_dir=log_dir, maps_dir=maps_dir, alpha = 0.6, beta = 0.4) #Run ACO training

Test Pheromone Map

In [ ]:
#Initialize the environment for testing
env = mywayhome_VZG(config_path=doomfinder("my_way_home_4_dir.cfg"), pheromone_evaporation_rate=0, render=True, frameskip=4) 

#Run the test function
test_saved_maps(env=env, num_tests=5, pheromone_grid=saved_pheromone_grid, alpha = 0.7, beta = 0.3) #Test the saved pheromone maps

Test Movement, making a function to predict acceleration 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

#Define the functions to fit the curve to

def exponential_decay(x, a, b):
    return a * (1 - np.exp(-b * x))

def tanh(x, a, b, c):
    return a * np.tanh(b * (x - c))

def log_func(x, a, b):
    return a * np.log(x) + b

function_map = {
    "Exponential Decay": (exponential_decay, ["a", "b"]),
    #"Logarithmic": (log_func, ["a", "b"]),
    #"Hyperbolic Tangent": (tanh, ["a", "b", "c"]),
}

def graph(steps, velocities, color, motion, fit_type):
    """
    Fit a curve to the data and plot the results.

    Args:
        steps (list): List of steps.
        velocities (list): List of velocities.
        color (str): Color for the data points.
        motion (str): Description of the motion.
        fit_type (str): Type of curve to fit.
    """
    
    #Convert to np arrays
    steps = np.array(steps) 
    velocities = np.array(velocities)

    #Select the fitting function
    func, param_names = function_map[fit_type]

    #Fit the curve
    try:
        popt, pcov = curve_fit(func, steps, velocities)
    except Exception as e:
        print(f"Failed to fit {fit_type}: {e}")
        return

    #Extract parameters
    params = ", ".join(f"{name}={val:.3f}" for name, val in zip(param_names, popt))

    #Generate points for the fitted curve
    x_fit = np.linspace(min(steps), max(steps), 1000)
    y_fit = func(x_fit, *popt)

    #Print the parameters
    print(f"Fitted parameters ({fit_type}/{motion}): {params}")

    #Plot the data and the fit
    plt.scatter(steps, velocities, label="Data", color=color)
    plt.plot(x_fit, y_fit, label=f"Fit: {fit_type} ({params})", color="red")
    plt.xlabel("Steps")
    plt.ylabel("Velocity")
    plt.legend()
    plt.title(f"Doom Acceleration {fit_type} Fit - {motion}")
    plt.grid()
    plt.show()


#Initialize the environment for testing
env = mywayhome_VZG(config_path=doomfinder("movement_test.cfg"), pheromone_evaporation_rate=0, render=False, frameskip=1) 

#Data collection
actions = [0, 1, 2, 3]
colors = ["blue", "orange", "green", "purple"]
titles = ["Moving Forward", "Moving Left", "Moving Right", "Moving Backward"]

for fit_type in function_map.keys():
    for action in actions:
        steps = []
        velocities = []
        color = colors[action]
        motion = titles[action]
        for i in range(1, 201):
            env.step(action = action)
            #time.sleep(0.9)
            steps.append(i)
            if action == 0 or action == 3:
                velocities.append(env.velocity_y)
            else:
                velocities.append(env.velocity_x)

            #print(f"step {i}, x velocity = {env.velocity_x}, y velocity = {env.velocity_y})")
        env.reset()

        graph(steps, velocities, color, motion, fit_type)

#Exponential decay seems to fit best

The issue is that this assumes the agent goes to a fullstop before changing directions, we can't assume this, but we compare with a simple friction function, and find that it matches down down to a tolerance of 1-e3. This is very close.

The reason we need to know this is that we need to predict movement of the agent for decision making, but due to frameskip, we don't always know the velocity, so we have to adjust it overtime. 

In [ ]:
def compare(a, b, steps, velocities, title):
    """
    Compare the exponential decay model and adjusted friction model to velocity data.

    Args:
        a (float): The exponential decay function approaches this value (asymptote).
        b (float): The rate at which the function approaches a.
        steps (array-like): The steps (time points) for the observed data.
        velocities (array-like): The observed velocity data points.
        title (str): The title for the motion being analyzed.
    """

    friction = 0.90625  #DOOM friction

    #Our friction floor/ceiling values should be the same as our exponential decay floor/ceiling values
    v_ceiling = abs(a) #Must be positive, defined by the exponential decay function ceiling/floor
    v_floor = 0 #Friction decays to zero

    v0 = v_ceiling #Initial velocity, will decay to 0 as exponential decay function reaches a

    #Time steps for friction model
    time_steps = np.arange(0, 200, 1)

    #Adjusted friction-based velocity model with floor and ceiling
    friction_velocities_adjusted = [v0]
    for t in time_steps[1:]:
        v_prev = friction_velocities_adjusted[-1]
        if v_prev > v_floor:  # Decaying toward floor
            v_next = v_floor + (v_prev - v_floor) * friction
        elif v_prev < v_ceiling:  # Decaying toward ceiling
            v_next = v_ceiling + (v_prev - v_ceiling) * friction
        friction_velocities_adjusted.append(v_next)

    #Exponential decay model
    exp_decay_velocities = a * (1 - np.exp(-b * time_steps))

    #Determine when friction approaches zero
    tolerance = 1e-3
    friction_zero_time = next((t for t, v in enumerate(friction_velocities_adjusted) if np.isclose(v, v_floor, atol=tolerance)), None)

    #Determine when exponential decay approaches a
    decay_target_velocity = a
    decay_target_time = next((t for t, v in enumerate(exp_decay_velocities) if np.isclose(v, decay_target_velocity, atol=tolerance)), None)

    #Plot observed data
    plt.scatter(steps, velocities, label="Observed Data", color="purple", alpha=0.7)

    #Add vertical lines for the two events
    if friction_zero_time is not None:
        plt.axvline(friction_zero_time, color="blue", linestyle=":", label=f"Friction hits 0 (t={friction_zero_time})", alpha=0.7)
    if decay_target_time is not None:
        plt.axvline(decay_target_time, color="red", linestyle=":", label=f"Decay hits a (t={decay_target_time})", alpha=0.7)

    #Plot both models
    plt.plot(time_steps, friction_velocities_adjusted, label="Adjusted Friction Model (Floor & Ceiling)", color="blue")
    plt.plot(time_steps, exp_decay_velocities, label="Exponential Decay Model", color="red", linestyle="--")

    #Add labels, legend, and grid
    plt.xlabel("Time Steps")
    plt.ylabel("Velocity")
    plt.title(f"Comparison for {title}")
    plt.legend()
    plt.grid()
    plt.show()

#Initialize the environment for testing
env = mywayhome_VZG(config_path=doomfinder("movement_test.cfg"), pheromone_evaporation_rate=0, render=False, frameskip=1) 

#Data collection
step_sets = []
velocity_sets = []

for fit_type in function_map.keys():
    for action in actions:
        steps = []
        velocities = []
        color = colors[action]
        motion = titles[action]
        for i in range(1, 201):
            env.step(action=action)
            steps.append(i)
            if action == 0 or action == 3:
                velocities.append(env.velocity_y)
            else:
                velocities.append(env.velocity_x)
        env.reset()
        step_sets.append(np.array(steps))
        velocity_sets.append(np.array(velocities))

#Comparing models using collected data
params = ((-7.552, 0.098), (7.250, 0.098), (-7.250, 0.098), (7.552, 0.098))  #Exponential decay parameters

for param, steps, velocities, title in zip(params, step_sets, velocity_sets, titles):
    print(f"Model for {title}: {param}")
    compare(*param, steps, velocities, title)